In [4]:
# Import necessary libraries
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from timm import create_model
import pickle
import random
from data_utils import permute_elements

/home/fodl/nimrodd_new/miniconda3/envs/req_test_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Utility functions:

## Function: load_and_permute_data
 This function loads data from the source folder, applies a random or custom permutation to the data,
 and saves the permuted data and labels to the destination folder.

In [5]:
def load_and_permute_data(src_folder, dst_folder, perm_path=None):
    # Generate a random permutation of the numbers from 0 to 1023 or load custom permutation
    if perm_path:
        with open(perm_path, 'rb') as f:
            random_permutation = pickle.load(f)
            print(f"Loaded permutation from {perm_path}")
    else:
        random_permutation = random.sample(range(1024), 1024)

    # List of data and label file names
    data_files = ["train_data.pt", "test_data.pt", "val_data.pt"]
    label_files = ["train_labels.pt", "test_labels.pt", "val_labels.pt"]

    # Load, permute and save data files
    for data_file in data_files:
        # Load the data tensor
        data_tensor = torch.load(os.path.join(src_folder, data_file))

        # Apply the permute_elements function
        permuted_data_tensor = permute_elements(data_tensor, random_permutation)

        # Save the permuted data tensor to the destination folder
        torch.save(permuted_data_tensor, os.path.join(dst_folder, data_file))

    # Load and save label files
    for label_file in label_files:
        # Load the label tensor
        label_tensor = torch.load(os.path.join(src_folder, label_file))

        # Save the label tensor to the destination folder
        torch.save(label_tensor, os.path.join(dst_folder, label_file))

# Step 1: Create a randomly permuted version of the dataset cifar10

In [6]:
src_folder = "user_datasets/cifar10_binary"
dst_folder = "user_datasets/cifar10_binary_rand_perm"
perm_path = None

if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)  # Deletes the contents of the folder

os.makedirs(dst_folder)  # Creates the folder

load_and_permute_data(src_folder, dst_folder, perm_path=perm_path)

# Step 2: Measure entropy for the different rearrangements

# Measuring: Original data
To do this, change the 'data_dir_name' field in the yaml config files 
'configs/cifar10_binary_measure_entropy_config.yaml'
to cifar10_binary. Then, run the following commands: 

In [24]:
!python3 measure_entropy.py configs/cifar10_binary_measure_entropy_config.yaml

In [27]:
save_add = "user_datasets/cifar10_binary/per_level_entanglement_entropies.pkl" # for surrogate change this to 'per_level_surrogate_entropies'
with open(save_add, 'rb') as f:
    entropy_measurement = pickle.load(f)
print(entropy_measurement)

{'level_1': 0.24817007035017014, 'level_2': 0.062260269245598465}


# Measuring: Randomly permuted data
To do this, change the 'data_dir_name' field in the yaml config files 
'configs/cifar10_binary_measure_entropy_config.yaml'
to cifar10_binary_rand_perm. Then, run the following commands: 

In [21]:
!python3 measure_entropy.py configs/cifar10_binary_measure_entropy_config.yaml

In [26]:
save_add = "user_datasets/cifar10_binary_rand_perm/per_level_entanglement_entropies.pkl"
with open(save_add, 'rb') as f:
    entropy_measurement = pickle.load(f)
print(entropy_measurement)

{'level_1': 0.41364074498414993, 'level_2': 0.08126604394055903}
